In [1]:
pwd

'/home/idies/workspace/Storage/sdowell/persistent/ALEdb/BenchmarkingFinetuning'

In [2]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import EsmForMaskedLM, EsmTokenizer
import re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from validate_TL_esm2Finetuning import *


# 1. Set up your parameters
model_path = model_path = "/home/idies/workspace/Storage/sdowell/persistent/ALEdb/runs/ESM_8M_test_1/checkpoint-5000"

# Example RNase III protein sequence from E. coli (replace with your actual sequence)
wild_type_seq = """\
MNPIVINRLQRKLGYTFNHQELLQQALTHR\
SASSKHNERLEFLGDSILSYVIANALYHRFP\
RVDEGDMSRMRATLVRGNTLAELAREFELGE\
CLRLGPGELKSGGFRRESILADTVEALIGGV\
FLDSDIQTVEKLILNWYQTRLDEISPGDKQKDP\
KTRLQEYLQGRHLPLPTYLVVQVRGEAHDQEFTI\
HCQVSGLSEPVVGTGSSRRKAEQAAAEQALKKLELE\
"""

# 2. Load your model
model, tokenizer, device = load_model(model_path)

# 3. Define some mutations to test (or load from your DMS dataset)
# Example - create mutations for a few positions
test_mutations = ["A45G", "D134E", "V22I"]

# Or load mutations from your DMS dataset
dms_data = pd.read_csv("RNC_ECOLI_Weeks_2023.csv")
dms_mutations = dms_data["mutant"].tolist()  # Adjust column name if needed

# Predict on a small subset first to test
sample_mutations = dms_mutations[:10]
print(sample_mutations)

2025-03-03 15:29:53.685625: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/runpy.py", line 87, i

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/site-packages/traitlets/config/application.

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/idies/miniconda3/envs/finetuning/lib/python3.9/site-packages/traitlets/config/application.

AttributeError: _UFUNC_API not found

ImportError: numpy.core.umath failed to import

RuntimeError: Failed to import transformers.models.esm.modeling_esm because of the following error (look up to see its traceback):
Unable to convert function return value to a Python type! The signature was
	() -> handle

In [3]:
predictions = predict_mutations(model, tokenizer, device, wild_type_seq, sample_mutations)
display(predictions)

Processing mutations:   0%|          | 0/10 [00:00<?, ?it/s]

mut_output is: 
 MaskedLMOutput(loss=None, logits=tensor([[[ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         ...,
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571]]]), hidden_states=None, attentions=None)
mut_output is: 
 MaskedLMOutput(loss=None, logits=tensor([[[ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         ...,
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.1215,  -6.6571],
         [ -8.6286,  -6.6569,  -7.6237,  ..., -16.1216, -16.12

,mutation,position,wild_type,mutant,predicted_score
0,A113C,113,A,C,0.0
1,A113D,113,A,D,0.0
2,A113E,113,A,E,0.0
3,A113F,113,A,F,0.0
4,A113G,113,A,G,0.0
5,A113H,113,A,H,0.0
6,A113I,113,A,I,0.0
7,A113K,113,A,K,0.0
8,A113L,113,A,L,0.0
9,A113M,113,A,M,0.0


In [8]:
# 4. To run predictions on all mutations (this may take a while)
#all_predictions = predict_mutations(model, tokenizer, device, wild_type_seq, dms_mutations)

# 5. Save the predictions
#all_predictions.to_csv("esm2_predictions.csv", index=False)

# 6. Validate against DMS data (if you have it)
import scipy.stats as stats

# Make sure column names match between your prediction and DMS data
correlation, comparison_df = validate_against_dms(all_predictions, dms_data)

# 7. Visualize the correlation
plt.figure(figsize=(10, 8))
sns.scatterplot(x='DMS_score', y='predicted_score', data=comparison_df)
plt.title(f'ESM2 Predictions vs DMS Experimental Scores (Spearman ρ = {correlation:.3f})')
plt.xlabel('Experimental DMS Score')
plt.ylabel('Predicted ESM2 Score')
plt.grid(alpha=0.3)
plt.show()

# 8. Analyze prediction performance by region or amino acid type
# For example, let's see if predictions are better for certain amino acid changes
comparison_df['wt_aa_type'] = comparison_df['wild_type'].map({
    'A': 'hydrophobic', 'V': 'hydrophobic', 'L': 'hydrophobic', 'I': 'hydrophobic',
    'M': 'hydrophobic', 'F': 'hydrophobic', 'W': 'hydrophobic', 'P': 'hydrophobic',
    'D': 'charged', 'E': 'charged', 'K': 'charged', 'R': 'charged', 'H': 'charged',
    'S': 'polar', 'T': 'polar', 'N': 'polar', 'Q': 'polar', 'Y': 'polar', 'C': 'polar',
    'G': 'special'
})

# Group by amino acid type and calculate correlations
grouped = comparison_df.groupby('wt_aa_type')
for name, group in grouped:
    corr, _ = stats.spearmanr(group['predicted_score'], group['DMS_score'])
    print(f"Correlation for {name} amino acids: {corr:.3f} (n={len(group)})")#

Processing mutations:   0%|          | 0/10 [00:00<?, ?it/s]

,mutation,position,wild_type,mutant,predicted_score
0,A113C,113,A,C,0.0
1,A113D,113,A,D,0.0
2,A113E,113,A,E,0.0
3,A113F,113,A,F,0.0
4,A113G,113,A,G,0.0
5,A113H,113,A,H,0.0
6,A113I,113,A,I,0.0
7,A113K,113,A,K,0.0
8,A113L,113,A,L,0.0
9,A113M,113,A,M,0.0


NameError: name 'stats' is not defined

In [6]:
import scipy.stats as stats
print(stats.spearmanr([1, 2, 3], [3, 2, 1]))


SpearmanrResult(correlation=-1.0, pvalue=0.0)


In [4]:
def analyze_model_weights(model):
    """Analyze if model weights have meaningful variation"""
    
    # Check embedding layer
    if hasattr(model, 'esm'):
        # ESM2 structure
        embedding_weight = model.esm.embeddings.word_embeddings.weight
    else:
        # Generic structure
        embedding_layers = [m for name, m in model.named_modules() if 'embedding' in name.lower()]
        if embedding_layers:
            embedding_weight = list(embedding_layers[0].parameters())[0]
        else:
            print("Could not locate embedding layer")
            return
    
    # Check statistics of embedding weights
    emb_mean = embedding_weight.mean().item()
    emb_std = embedding_weight.std().item()
    emb_min = embedding_weight.min().item()
    emb_max = embedding_weight.max().item()
    
    print(f"Embedding weight statistics:")
    print(f"  Mean: {emb_mean:.6f}")
    print(f"  Std Dev: {emb_std:.6f}")
    print(f"  Min: {emb_min:.6f}")
    print(f"  Max: {emb_max:.6f}")
    
    # A model with reasonable weights should have non-zero standard deviation
    if emb_std < 0.01:
        print("WARNING: Embedding weights have very low variance!")
    
    # Check if any parameters are NaN or Inf
    has_nan = False
    for name, param in model.named_parameters():
        if torch.isnan(param).any() or torch.isinf(param).any():
            print(f"WARNING: Parameter {name} contains NaN or Inf values!")
            has_nan = True
    
    if not has_nan:
        print("All parameters contain valid values (no NaN/Inf)")
    
    # Check attention layers
    attention_layers = [m for name, m in model.named_modules() 
                       if any(x in name.lower() for x in ['attention', 'self', 'mha'])]
    
    if attention_layers:
        print(f"\nFound {len(attention_layers)} attention-related layers")
        # Sample a few attention layers to check for identical weights
        for i, layer in enumerate(attention_layers[:3]):
            for name, param in layer.named_parameters():
                if 'weight' in name and param.dim() >= 2:
                    # Check if weights look identical across rows/columns
                    if param.size(0) > 1 and param.size(1) > 1:
                        same_rows = torch.allclose(param[0], param[1], rtol=1e-4)
                        if same_rows:
                            print(f"WARNING: Layer {i}, param {name} has identical rows!")
    else:
        print("Could not locate attention layers")

In [6]:
analyze_model_weights(model)

Embedding weight statistics:
  Mean: 0.000000
  Std Dev: 0.000000
  Min: 0.000000
  Max: 0.000000
All parameters contain valid values (no NaN/Inf)

Found 66 attention-related layers


In [7]:
from transformers import EsmForMaskedLM, EsmTokenizer

# Load the base model
base_model_name = "facebook/esm2_t6_8M_UR50D"
base_tokenizer = EsmTokenizer.from_pretrained(base_model_name)
base_model = EsmForMaskedLM.from_pretrained(base_model_name)

# Check embedding weights
embedding_weight = base_model.esm.embeddings.word_embeddings.weight
print(f"Base model embedding statistics:")
print(f"  Mean: {embedding_weight.mean().item():.6f}")
print(f"  Std Dev: {embedding_weight.std().item():.6f}")
print(f"  Min: {embedding_weight.min().item():.6f}")
print(f"  Max: {embedding_weight.max().item():.6f}")

Base model embedding statistics:
  Mean: -0.013621
  Std Dev: 0.138587
  Min: -2.035156
  Max: 1.202148
